# Hermes api

The hermes python api is a wrapper ont top of the Fortran library.

For it to be compiled you need to add:

options: api

in your systel.cfg file. This will also compile the Telemac-Mascaret api if you want only hermes ass

options: api only_hermes

The wrapper is contained into one class **HermesFile**. This class will alow you to manipulate file in the formats handled by Hermes (MED, SERAFIN, VTK, CGNS).

This is low-level class it makes basic operations if you want a more advanced class have a look at [TelemacFile](pretel/telemac_file.ipynb)

In [1]:
from telapy.api.hermes import HermesFile

help(HermesFile)

Help on class HermesFile in module telapy.api.hermes:

class HermesFile(builtins.object)
 |  HermesFile(file_name, fformat, access='r', boundary_file=None, log_lvl='INFO')
 |  
 |  The Generic Python class for TELEMAC-MASCARET APIs
 |  
 |  Methods defined here:
 |  
 |  __init__(self, file_name, fformat, access='r', boundary_file=None, log_lvl='INFO')
 |      Constructor for HermesFile
 |      
 |      @param file_name (string) Name of the file
 |      @param fformat (string) File format
 |      @param access (string) Access to the file ('r' for read 'w' for write)
 |      @param boundary_file (string) Name of the boundary file
 |      @param log_lvl (string) Logger level
 |  
 |  __repr__(self)
 |      representation of the object
 |  
 |  add_data(self, var_name, var_unit, time, record, first_var, values)
 |      Write inform ation for a given variable and a given timestep
 |      
 |      @param var_name Name of the variable
 |      @param var_unit Unit of the variable
 |      @par

The function you have access to can be divided in groups:
 * functions to read boundary information (**get\_bnd\_\***)
 * functions to read results information (**get\_data\_\***)
 * functions to read mesh information (**get\_mesh\_\***)
 * functions to write (**set\_header**, **write\_mesh**, **add\_data**, **set\_bnd**)
 
 Here is an example that read a couple informations from the file.
 
 If you add the boundary file at the initialisation of the calls you can access boundary file data as well (**get\_bnd\_value**).

In [2]:
from telapy.api.hermes import HermesFile
from os import path, environ

mesh_file = path.join(environ["HOMETEL"], 'examples', 'telemac2d', 'gouttedo', "geo_gouttedo.slf")
bnd_file = path.join(environ["HOMETEL"], 'examples', 'telemac2d', 'gouttedo', "geo_gouttedo.cli")
fformat = "SERAFIN"

# Opening file with hermes in read mode
myfile = HermesFile(mesh_file, fformat, 'r', boundary_file=bnd_file)

print("title of the file: ", myfile.get_mesh_title())
print("number of elements: ", myfile.get_mesh_nelem())
print("number of points: ", myfile.get_mesh_npoin())
print("number of time steps: ", myfile.get_data_ntimestep())
print("number of variables: ", myfile.get_data_nvar())
print("number of boundary points: ", myfile.get_bnd_npoin())
# time step starts from 0
record = 0
# The function will return as a numpy array the value of MAILLAGE at time step 0
bottom = myfile.get_data_value("MAILLAGE", record)

print("bottom", bottom)

del myfile

title of the file:  TELEMAC 2D : GOUTTE D'EAU DANS UN BASSIN$                               SERAFIN 
number of elements:  8978
number of points:  4624
number of time steps:  1
number of variables:  1
number of boundary points:  268
bottom [ 0.         19.80000305 20.1000061  ... 19.79998779 19.79998779
  0.30000007]


Here is an example where we will read data from one file and create a new file with things modified.

For that we use the function **import_from** that reads all the data from a file and writes it into another.

You would be able to change the output format by changing the value of fformat for the output file.

In [3]:
from telapy.api.hermes import HermesFile
from os import path, environ, remove

mesh_file = path.join(environ["HOMETEL"], 'examples', 'telemac2d', 'gouttedo', "geo_gouttedo.slf")
fformat = "SERAFIN"

# Opening file with hermes in read mode
myfile = HermesFile(mesh_file, fformat, 'r')
    
root, ext = path.splitext(mesh_file)
mesh_file2 = path.join(path.dirname(mesh_file), 'geo_gouttedo_new_bottom.slf')

if path.exists(mesh_file2):
    remove(mesh_file2)

# Opening new file in write mode
myfile2 = HermesFile(mesh_file2, fformat, 'w')

# Importing all the info from mesh_file into the class
myfile2.import_from(myfile)
    
bottom = myfile2.get_data_value("MAILLAGE", 0)

bottom += 20

myfile2.add_data("MAILLAGE", "M", 0.0, 0, True, bottom)

print(myfile)
print(myfile2)

# Deleting the object will close the file
del myfile
del myfile2

********************************
Generic info
********************************
Title: ELEMAC 2D : GOUTTE D'EAU DANS UN BASSIN$                               
Date: 0/0/0 0H0M0S
********************************
Mesh info
********************************
Ndim: 2
Element type: TRIANGLE
Npoin: 4624
Nelem: 8978
Ndp: 3
nplan: 0
coordinates:
 - On x :[ 0.         19.80000305 20.1000061  ... 19.79998779 19.79998779
  0.30000007]
 - On y :[ 0.          0.          0.         ... 19.79998779  0.30000007
 19.79998779]
ikle: [[4438 4437 4371]
 [4440 4439 4373]
 [ 447  513  446]
 ...
 [  10  270    9]
 [ 267  266 4620]
 [ 455  521  454]]
********************************
Parallel info
********************************
No parallel information
********************************
Bnd info
********************************
No boundary information********************************
Data info
********************************
ntimestep: 1
nvar: 1
var info:
('MAILLAGE', '')
Time: 0.000000s
 - for MAILLAGE:
     [ 0

## Logging feature

When creating a HermesFile you can define the logging level (argument log_lvl).

This will setting how verbose the code should be.

By default it will not display much.

Below is the first example of reading information by with a more verbose option.

In [4]:
from telapy.api.hermes import HermesFile
from os import path, environ

# Not necessary in your code this is just to force the notebook to display the logging info
import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
####

mesh_file = path.join(environ["HOMETEL"], 'examples', 'telemac2d', 'gouttedo', "geo_gouttedo.slf")
bnd_file = path.join(environ["HOMETEL"], 'examples', 'telemac2d', 'gouttedo', "geo_gouttedo.cli")
fformat = "SERAFIN"

# Opening file with hermes in read mode
myfile = HermesFile(mesh_file, fformat, 'r', boundary_file=bnd_file, log_lvl='DEBUG')

print("title of the file: ", myfile.get_mesh_title())
print("number of elements: ", myfile.get_mesh_nelem())
print("number of points: ", myfile.get_mesh_npoin())
print("number of time steps: ", myfile.get_data_ntimestep())
print("number of variables: ", myfile.get_data_nvar())
print("number of boundary points: ", myfile.get_bnd_npoin())
# time step starts from 0
record = 0
# The function will return as a numpy array the value of MAILLAGE at time step 0
bottom = myfile.get_data_value("MAILLAGE", record)

print("bottom", bottom)

del myfile

DEBUG:telapy.api.hermes:Loading hermes f2py module


DEBUG:telapy.api.hermes:Opening mesh /home/d60881/telemac-mascaret/examples/telemac2d/gouttedo/geo_gouttedo.slf in format b'SERAFIN ' in mode b'READ     '


DEBUG:telapy.api.hermes:Opening bnd b'SERAFIN ' in format /home/d60881/telemac-mascaret/examples/telemac2d/gouttedo/geo_gouttedo.slf in mode b'READ     '


DEBUG:telapy.api.hermes:Getting number of dimension


DEBUG:telapy.api.hermes:Getting number of elements


...

DEBUG:telapy.api.hermes:Get number of variables


DEBUG:telapy.api.hermes:Getting number of boundary points


DEBUG:telapy.api.hermes:Getting number of points 10 10


DEBUG:telapy.api.hermes:Getting data for MAILLAGE         at record 0


title of the file:  TELEMAC 2D : GOUTTE D'EAU DANS UN BASSIN$                               SERAFIN 
number of elements:  8978
number of points:  4624
number of time steps:  1
number of variables:  1
number of boundary points:  268
bottom [ 0.         19.80000305 20.1000061  ... 19.79998779 19.79998779
  0.30000007]
